In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys,os
import pandas as pd

In [ ]:
hw_path=os.path.abspath('../hw')
if hw_path not in sys.path:
    sys.path.append(hw_path)
from dvl import parse_line

In [ ]:
#data_file='../../data/221008-112305/viewer_data.pkl'
#data_file='../../data/221007-111613/viewer_data.pkl'
data_file='../../data/221004-160135/viewer_data.pkl' #rope in the marina
#fd=open(data_file,'rb')
!ls -all -h $data_file

In [ ]:
keys=set()
dvl_data_raw=[]
with open(data_file,'rb') as fd:
    while 1:
        try:
            d=pickle.load(fd)
            keys.add(d[0])
            if d[0]==b'topic_dvl_raw':
                dvl_data_raw.append(d[1:])
        except EOFError:
            break
print(keys)
    

In [ ]:
#parse_line(dvl_data_raw[0])
ts_vel=[]
ts_pos=[]
vel=[]
covs=[]
foms=[]
pos=[]
tr_vels=[[],[],[],[]]
tr_dists=[[],[],[],[]]
tr_rssi=[[],[],[],[]]
tr_nsd=[[],[],[],[]]
tr_ts=[]
for d in dvl_data_raw:
    m=parse_line(d[0]['dvl_raw'])
    if m and m['type'] in ['vel']:#,'deadreacon']:
        ts_vel.append(m['tov']/1e6)
        vel.append((m['vx'],m['vy'],m['vz']))
        covs.append(m['cov'])
        foms.append(m['fom'])
        if m['status']>0:
            print(m)
    if m and m['type'] in ['deadreacon']:
        ts_pos.append(m['time'])
        pos.append((m['x'],m['y'],m['z']))
    if m and m['type'] in ['transducer2']:
        #print(m)
        tr_vels[int(m['id'])].append(m['velocity'])
        tr_dists[int(m['id'])].append(m['distance'])
        tr_rssi[int(m['id'])].append(m['rssi'])
        tr_nsd[int(m['id'])].append(m['nsd'])
        #ts_pos.append(m['time'])
        ts=d[0]['ts']
        if m['id']==0:
            tr_ts.append(ts)
        #pos.append((m['x'],m['y'],m['z']))

ts_vel=np.array(ts_vel)
vel=np.array(vel)
covs=np.array(covs)
ts_pos=np.array(ts_pos)
start=ts_pos[0]
ts_pos-=start
ts_vel-=start
pos=np.array(pos)
foms=np.array(foms)
tr_vels=np.array(tr_vels).T
tr_dists=np.array(tr_dists).T
tr_rssi=np.array(tr_rssi).T
tr_nsd=np.array(tr_nsd).T
tr_ts=np.array(tr_ts)
tr_ts-=tr_ts[0]

In [ ]:
plt.figure()
#plt.plot(ts_vel,np.cumsum(vel[:,1]),'+-')
tresh=0.01
inds=np.nonzero(covs[:,0+3*0]<tresh)[0]
for i in [1,2]:
    inds=np.intersect1d(inds,np.nonzero(covs[:,i+3*i]<tresh)[0])
#plt.plot(ts_vel[inds],vel[inds,:],'+-',alpha=0.4)
for i in [0,1,2]:
    #plt.plot(ts_vel[inds],vel[inds,i],'+-'+'rgb'[i],alpha=0.4)
    plt.plot(ts_vel[inds],covs[inds,i+3*i]*1e4,'.-'+'rgb'[i],alpha=0.1,)

    #plt.plot(ts_vel[inds],vel[inds,i],'+-'+'rgb'[i],alpha=0.4)
plt.plot(ts_vel[inds],foms[inds]*1e1,'.-m',alpha=0.1,)

#inds

In [ ]:
plt.figure()
ind=1
plt.plot(ts_vel[inds][1:],np.cumsum(vel[inds][1:,ind])*np.diff(ts_vel[inds]),'+-b')
plt.plot(ts_pos,pos[:,ind],'+-r')


In [ ]:
plt.figure()
for i in range(4):
    plt.plot(tr_ts,tr_vels[:,i],'-+'+'rgbm'[i],alpha=0.2)

In [ ]:
plt.figure('transducer')
ax=plt.subplot(3,1,1)
plt.title('range')
for i in range(4):
    plt.plot(tr_ts,tr_dists[:,i],'-+'+'rgbm'[i],alpha=0.2)
plt.subplot(3,1,2,sharex=ax)
plt.title('rssi')
for i in range(4):
    plt.plot(tr_ts,tr_rssi[:,i],'-+'+'rgbm'[i],alpha=0.1)
plt.subplot(3,1,3,sharex=ax)
plt.title('nsd')
for i in range(4):
    plt.plot(tr_ts,tr_nsd[:,i],'-+'+'rgbm'[i],alpha=0.2)
